In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from dateutil import parser
%matplotlib inline

from sklearn.model_selection import train_test_split

from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.model_selection import GridSearchCV
import pickle
from lightgbm import LGBMClassifier
print('Library added')

Library added


In [29]:
data = 'data.csv'

In [30]:
df = pd.read_csv(data)
df.shape

(303, 14)

In [31]:
df.isnull().sum()

age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [32]:
cols = df.columns
cols

Index(['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach',
       'exang', 'oldpeak', 'slope', 'ca', 'thal', 'target'],
      dtype='object')

In [33]:
print("# rows in dataset {0}".format(len(df)))
print("----------------------------------------")

for col in cols:
    print("# rows in {1} with zero values: {0}".format(len(df.loc[df[col] == 0]), col))

# rows in dataset 303
----------------------------------------
# rows in age with zero values: 0
# rows in sex with zero values: 96
# rows in cp with zero values: 143
# rows in trestbps with zero values: 0
# rows in chol with zero values: 0
# rows in fbs with zero values: 258
# rows in restecg with zero values: 147
# rows in thalach with zero values: 0
# rows in exang with zero values: 204
# rows in oldpeak with zero values: 99
# rows in slope with zero values: 21
# rows in ca with zero values: 175
# rows in thal with zero values: 2
# rows in target with zero values: 138


In [34]:
final_cols=cols
final_cols=list(final_cols)
final_cols.remove('sex')
final_cols.remove('target')
final_cols.remove('age')
final_cols

['cp',
 'trestbps',
 'chol',
 'fbs',
 'restecg',
 'thalach',
 'exang',
 'oldpeak',
 'slope',
 'ca',
 'thal']

# Feature engineering

In [35]:
df.target.value_counts()

1    165
0    138
Name: target, dtype: int64

In [44]:
X = df
y = df.target

from imblearn.over_sampling import SMOTE
sm = SMOTE(random_state=42)
X_res_OS, Y_res_OS = sm.fit_resample(X, y)
print(X_res_OS)

X_train, X_test, y_train, y_test = train_test_split(X_res_OS[final_cols], Y_res_OS, test_size=.1, random_state=10)

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang   oldpeak  \
0     63    1   3       145   233    1        0      150      0  2.300000   
1     37    1   2       130   250    0        1      187      0  3.500000   
2     41    0   1       130   204    0        0      172      0  1.400000   
3     56    1   1       120   236    0        1      178      0  0.800000   
4     57    0   0       120   354    0        1      163      1  0.600000   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...       ...   
325   63    1   0       143   210    0        0      133      0  1.975897   
326   61    0   0       175   256    0        0      145      1  0.506778   
327   58    0   0       139   374    0        0      140      0  2.292009   
328   54    1   0       127   254    0        1      161      0  0.033361   
329   57    1   0       128   258    0        0      132      1  2.583925   

     slope  ca  thal  target  
0        0   0     1       1  
1        0   

In [37]:
Y_res_OS.value_counts()

1    165
0    165
Name: target, dtype: int64

In [38]:
# from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
fill = SimpleImputer(missing_values = 0, strategy='mean')

X_train = fill.fit_transform(X_train)
X_test = fill.fit_transform(X_test)

In [39]:
def FitModel(X_train, y_train, X_test, y_test, algo_name, algorithm, gridSearchParams, cv):
    
    np.random.seed(10)
    
    grid = GridSearchCV(
        estimator=algorithm,
        param_grid=gridSearchParams,
        cv=cv, scoring='accuracy', verbose=1, n_jobs=-1
    )
    
    grid_result = grid.fit(X_train, y_train)
    best_params = grid_result.best_params_
    pred = grid_result.predict(X_test)
    cm = confusion_matrix(y_test, pred)
    
    print(pred)
    pickle.dump(grid_result, open(algo_name, 'wb'))
    
    print('Best Params :', best_params)
    print('Classification Report', classification_report(y_test, pred))
    print('Accuracy Score', str(accuracy_score(y_test, pred)))
    print('Confusion Matrix: \n', cm)

# Logistic Regression

In [40]:
penalty = ['l1', 'l2']

C = np.logspace(0, 4, 10)

hyperparameters = dict(C=C, penalty=penalty)

FitModel(X_train, y_train, X_test, y_test, 'LogisticRegression', LogisticRegression(), hyperparameters, cv=5)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


[1 1 0 0 0 1 1 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 1 0 0 0 1 0 0 1 1 0 1]
Best Params : {'C': 1.0, 'penalty': 'l2'}
Classification Report               precision    recall  f1-score   support

           0       0.61      0.85      0.71        13
           1       0.87      0.65      0.74        20

    accuracy                           0.73        33
   macro avg       0.74      0.75      0.73        33
weighted avg       0.77      0.73      0.73        33

Accuracy Score 0.7272727272727273
Confusion Matrix: 
 [[11  2]
 [ 7 13]]


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    1.3s finished
/Users/sannykumar/Desktop/cosmicML/env/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


# Xgboost

In [41]:
param = {
    'n_estimators': [100, 500, 1000, 1500, 2000],
    'max_depth': [2, 3, 4, 5, 6, 7],
    'learning_rate': np.arange(0.01, 0.1, 0.01).tolist()
}

FitModel(X_train, y_train, X_test, y_test, 'XGBoost', XGBClassifier(), param, cv=5)

Fitting 5 folds for each of 270 candidates, totalling 1350 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:   10.4s
[Parallel(n_jobs=-1)]: Done 208 tasks      | elapsed:   48.8s
[Parallel(n_jobs=-1)]: Done 458 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 808 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1258 tasks      | elapsed:  4.2min


[1 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 1 0 1 1 0 1 1 0 1 1 1 0 0 1 1 1 1]
Best Params : {'learning_rate': 0.02, 'max_depth': 2, 'n_estimators': 100}
Classification Report               precision    recall  f1-score   support

           0       0.92      0.92      0.92        13
           1       0.95      0.95      0.95        20

    accuracy                           0.94        33
   macro avg       0.94      0.94      0.94        33
weighted avg       0.94      0.94      0.94        33

Accuracy Score 0.9393939393939394
Confusion Matrix: 
 [[12  1]
 [ 1 19]]


[Parallel(n_jobs=-1)]: Done 1350 out of 1350 | elapsed:  4.4min finished


# Random Forest

In [42]:
param = {
    'n_estimators': [100, 500, 1000, 1500, 2000],
    'max_depth': [2,3,4,5,6,7],
}
FitModel(X_train, y_train, X_test, y_test, 'Random Forest', RandomForestClassifier(), param, cv=5)

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  6.3min finished


[1 1 1 0 1 1 0 1 1 0 1 0 0 0 0 1 1 0 0 1 0 1 1 1 1 1 1 0 0 1 1 1 1]
Best Params : {'max_depth': 4, 'n_estimators': 100}
Classification Report               precision    recall  f1-score   support

           0       0.83      0.77      0.80        13
           1       0.86      0.90      0.88        20

    accuracy                           0.85        33
   macro avg       0.85      0.83      0.84        33
weighted avg       0.85      0.85      0.85        33

Accuracy Score 0.8484848484848485
Confusion Matrix: 
 [[10  3]
 [ 2 18]]


# SVC

In [26]:
param = {
    'C': [0.1, 1, 100, 1000],
    'gamma': [0.0001, 0.001, 0.005, 0.1, 1, 3, 5]
}

FitModel(X_train, y_train, X_test, y_test, 'SVC', SVC(), param, cv=5)

Fitting 5 folds for each of 28 candidates, totalling 140 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  56 tasks      | elapsed:    3.2s


[1 1 1 0 1 1 0 0 1 0 1 0 0 0 0 1 0 0 1 1 0 1 1 1 0 0 1 0 0 1 1 1 1]
Best Params : {'C': 100, 'gamma': 0.0001}
Classification Report               precision    recall  f1-score   support

           0       0.80      0.92      0.86        13
           1       0.94      0.85      0.89        20

    accuracy                           0.88        33
   macro avg       0.87      0.89      0.88        33
weighted avg       0.89      0.88      0.88        33

Accuracy Score 0.8787878787878788
Confusion Matrix: 
 [[12  1]
 [ 3 17]]


[Parallel(n_jobs=-1)]: Done 140 out of 140 | elapsed:    3.7s finished


AttributeError: 'numpy.ndarray' object has no attribute 'cols'